In [5]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import re
from openai import OpenAI
import base64
import os
from IPython.display import display
from PIL import Image
import io
import pandas as pd

In [6]:
# 目標網址
url = f"https://selcrs.nsysu.edu.tw/menu1/qrycourse.asp?HIS=2&eng=&in_eng=&IDNO=&ITEM="

# 建立seesion 確保待會發送驗證碼都會在同一個session
session = requests.Session()

# 發送 GET 請求
response = session.get(url, verify=False)

# 確認回應成功
if response.status_code == 200:
    response.encoding = "utf-8"
    html = response.text
    pprint(html)
else:
    print("錯誤，Status Code:", response.status_code)

c:\Users\User\github\llm_tutorial\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'selcrs.nsysu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


('<html>\r\n'
 '\r\n'
 '\r\n'
 '<head>\r\n'
 '<meta http-equiv="Content-Language" content="zh-tw">\r\n'
 '<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n'
 '<title>查詢課程資料</title>\r\n'
 '<meta name="Microsoft Theme" content="none, default">\r\n'
 '<meta name="Microsoft Border" content="none, default">\r\n'
 '<style type="text/css">\r\n'
 '<!--\r\n'
 '.unnamed1 {  font-size: 9pt}\r\n'
 '.small { font-size: 9pt}\r\n'
 '.unnamed2 {  font-size: 12pt}\r\n'
 '.s10 {  font-size: 10pt}\r\n'
 '-->\r\n'
 '<!--\r\n'
 '.font{  font-size: 9pt}\r\n'
 'A:link  {color: rgb(0,0,255); font-style: normal; text-decoration: none; '
 'cursor: hand}\r\n'
 'A:visited  {color: rgb(0,0,255); font-style: normal; text-decoration: '
 'none}\r\n'
 'A:active  {color:  rgb(255,0,0); font-style: normal; text-decoration: '
 'none}\r\n'
 'A:hover {color: rgb(255,0,0); font-style:bold; text-decoration: none}\r\n'
 '.QGCellName { FONT-SIZE: 9pt; COLOR: 0000FF; }\r\n'
 '.QGCell { FONT-SIZE: 9pt; TEXT

In [7]:
# 從 HTML 找出驗證碼圖片 URL
match = re.search(r'src="(validcode\.asp\?epoch=\d+)"', html)
if match:
    vc_url = "https://selcrs.nsysu.edu.tw/menu1/" + match.group(1)
    print("驗證碼圖片網址:", vc_url)

    # 取得圖片
    img_res = session.get(vc_url, verify=False)

    # 儲存圖片
    img = Image.open(io.BytesIO(img_res.content))
    img.save("captcha.png", format="PNG")
else:
    print("找不到驗證碼網址")

驗證碼圖片網址: https://selcrs.nsysu.edu.tw/menu1/validcode.asp?epoch=1763934383


c:\Users\User\github\llm_tutorial\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'selcrs.nsysu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [8]:
# 初始化 OpenAI Client
client = OpenAI()

# 將圖片轉為 base64
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

image_path = "captcha.png"  # 圖片位置
b64_image = encode_image(image_path)

prompt = "請問圖片內的數字是什麼?共4位。你只要回答我結果，不要有多餘文字說明。"

response = client.responses.create(
    model="gpt-4o",  # 支援 vision 的模型
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {
                    "type": "input_image",
                    "image_url": f"data:image/png;base64,{b64_image}",  # 這裡是重點
                },
            ],
        }
    ],
)

In [9]:
# 辨識結果
openaiRes = response.output[0].content[0].text
match = re.search(r"\d{4}", openaiRes)
if match:
    captchaRes = match.group(0)
    print("驗證碼：", captchaRes)
else:
    print("找不到 4 位數字")

display(Image.open("captcha.png"))

驗證碼： 3696


In [10]:
# 建立 Payload
payload = {
    "HIS": "2",
    "IDNO": "",
    "ITEM": "",
    "D0": "1141",
    "DEG_COD": "M",
    "D1": "M403",
    "D2": "",
    "CLASS_COD": "",
    "SECT_COD": "",
    "TYP": "1",
    "SDG_COD": "",
    "teacher": "",
    "crsname": "",
    "T3": "",
    "WKDAY": "",
    "SECT": "",
    "nowhis": "1",
    "ValidCode": captchaRes,
    "B1": "開始查詢",
}

# 發送 POST 請求
url2 = "https://selcrs.nsysu.edu.tw/menu1/dplycourse.asp?eng="
response = session.post(url2, data=payload, verify=False)

print("Status Code:", response.status_code)

# 確認回應成功
if response.status_code == 200:
    response.encoding = "utf-8"
    pprint(response.text)
else:
    print("錯誤，Status Code:", response.status_code)

c:\Users\User\github\llm_tutorial\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'selcrs.nsysu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Status Code: 200
('<html>\r'
 '<head>\r'
 '<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r'
 '<title>查詢</title>\r'
 '</head>\r'
 '<style type=text/CSS>\r'
 '<!--\r'
 ' BODY {\tSCROLLBAR-FACE-COLOR: #69668E;SCROLLBAR-ARROW-COLOR: #FFFFFF; '
 'SCROLLBAR-TRACK-COLOR: #B7B9CE; SCROLLBAR-SHADOW-COLOR: white; '
 'SCROLLBAR-3DLIGHT-COLOR: #69668E; ; background-attachment: fixed;\r'
 '  background-repeat: no-repeat; background-position: right top} \r'
 'a:link {color:#000080text-decoration: underline cursor: hand}\r'
 'a:visited,a:active {color:#0080C0text-decoration: underline cursor: hand}\r'
 'a:hover{color:rgb(255,93,0);text-decoration:none}\r'
 'td {font-size:12pt}\r'
 'select {font-size:9pt;font-color:#0033CC;line-height:140%}\r'
 '.head {font-size:9pt}\r'
 '.foot-chn {font-size:9pt;line-height:120%}\r'
 ".foot-eng {font-size:9px;font-family:'verdana'; letter-spacing:2px}\r"
 '.adduser  {font-size:9pt;line-height:120%}\r'
 '-->\r'
 '</style>\r'
 '<table width="100%"

In [11]:
# bs4解析網頁
soup = BeautifulSoup(response.text, "html.parser")

# 取出所需資料
tds = soup.find_all("td")
res = list()
for td in tds:
    res.append(td.text)
res = res[23:-2]

In [13]:
# 轉為Df
chunk_size = 21
rows = [res[i:i+chunk_size] for i in range(0, len(res), chunk_size)]
output = pd.DataFrame(rows)
output

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,114,1,財管碩,FM506,1,不分班,金融市場理論與實務FINANCIAL MARKETS THEORY AND PRACTICE,3,學期,選修,...,邱敬貿,"一8,6,7(管CM 1023)",678,,,,,,,《講授類》 金融工程學程金融科技學程財務管理專業學程
1,114,1,財管碩,FM510,1,不分班,財務管理理論FINANCIAL MANAGEMENT THEORIES,3,學期,選修,...,廖源星,"一2,3,4(管CM 3019)",234,,,,,,,《講授類》 金融科技學程財務管理專業學程
2,114,1,財管碩,FM517,1,不分班,現代投資組合理論與策略MODERN PORTFOLIO THEORY AND INVESTM...,3,學期,選修,...,"蕭珮婕,鄭義","四2,3,4(管CM 1025)",,,,234,,,,《講授類》 金融工程學程金融科技學程人工智能投資微學程財務管理專業學程
3,114,1,財管碩,FM521,1,不分班,財務研究方法（一）METHODOLOGY FOR FINANCE (I),3,學期,選修,...,佘博文,"五2,3,4(管CM 2037)",,,,,234,,,《講授類》 金融科技學程
4,114,1,財管碩,FM527,1,不分班,證券市場微結構實務SECURITIES MARKETS MICROSTRUCTURE PRA...,3,學期,選修,...,"邱敬貿,馬黛","三5,6,7(管CM 2020)",,,567,,,,,《講授類》 金融工程學程金融科技學程金融科技與創新微學程
5,114,1,財管碩,FM532,1,不分班,財務報表分析與權益評價FINANCIAL STATEMENT ANALYSIS AND EQ...,3,學期,選修,...,吳藺芝,"二8,6,7(管CM 3017)",,678,,,,,,《講授類》 ※英語授課
6,114,1,財管碩,FM537,1,不分班,投資銀行與法金實務INVESTMENT BANKING AND CORPORATE FINA...,3,學期,選修,...,"翁培師,蔡秉真","二5,6,7(管CM 3019)",,567,,,,,,《講授類》
7,114,1,財管碩,FM541,1,不分班,行為財務學的應用APPLICATIONS OF BEHAVIORAL FINANCE,3,學期,選修,...,蔡佳芬,"三2,3,4(管CM 3022)",,,234,,,,,《講授類》 ※英語授課
8,114,1,財管碩,FM548,1,不分班,企業社會責任CORPORATE SOCIAL RESPONSIBILITY,3,學期,選修,...,封之遠,"四5,6,7(管CM 2026)",,,,567,,,,《講授類》
9,114,1,財管碩,FM551,1,不分班,永續投資與綠色金融創新SUSTAINABLE INVESTMENT AND GREEN FI...,3,學期,選修,...,封之遠,"五8,6,7(管CM 2015)",,,,,678,,,《講授類》


In [15]:
# 建立Refer URL
def MakeReferUrl(page):
    return (
        f"https://selcrs.nsysu.edu.tw/menu1/dplycourse.asp?"
        f"a={payload['ValidCode']}"  # a= 就是驗證碼
        f"&D0={payload['D0']}"
        f"&DEG_COD={payload['DEG_COD']}"
        f"&D1={payload['D1']}"
        f"&D2={payload['D2']}"
        f"&CLASS_COD={payload['CLASS_COD']}"
        f"&SECT_COD={payload['SECT_COD']}"
        f"&TYP={payload['TYP']}"
        f"&SDG_COD={payload['SDG_COD']}"
        f"&teacher={payload['teacher']}"
        f"&crsname={payload['crsname']}"
        f"&T3={payload['T3']}"
        f"&WKDAY={payload['WKDAY']}"
        f"&SECT={payload['SECT']}"
        f"&HIS={payload['HIS']}"
        f"&IDNO={payload['IDNO']}"
        f"&ITEM={payload['ITEM']}"
        f"&bottom_per_page=10"
        f"&data_per_page=20"
        f"&page={page}"
    ) 

# 建立 Payload
headers = {
    "Referer": MakeReferUrl(page=1),
    "User-Agent": "Mozilla/5.0"
}

# 發送 POST 請求
targetPage = 2
url3 = MakeReferUrl(page=targetPage)
response = session.get(url3, headers=headers, verify=False)

print("Status Code:", response.status_code)

# 確認回應成功
if response.status_code == 200:
    response.encoding = "utf-8"
    pprint(response.text)
else:
    print("錯誤，Status Code:", response.status_code)

# bs4解析網頁
soup = BeautifulSoup(response.text, "html.parser")

# 取出所需資料
tds = soup.find_all("td")
res = list()
for td in tds:
    res.append(td.text)
res = res[23:-2]

# 轉為Df
chunk_size = 21
rows = [res[i:i+chunk_size] for i in range(0, len(res), chunk_size)]
iOutput = pd.DataFrame(rows)
iOutput

c:\Users\User\github\llm_tutorial\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'selcrs.nsysu.edu.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Status Code: 200
('<html>\r'
 '<head>\r'
 '<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r'
 '<title>查詢</title>\r'
 '</head>\r'
 '<style type=text/CSS>\r'
 '<!--\r'
 ' BODY {\tSCROLLBAR-FACE-COLOR: #69668E;SCROLLBAR-ARROW-COLOR: #FFFFFF; '
 'SCROLLBAR-TRACK-COLOR: #B7B9CE; SCROLLBAR-SHADOW-COLOR: white; '
 'SCROLLBAR-3DLIGHT-COLOR: #69668E; ; background-attachment: fixed;\r'
 '  background-repeat: no-repeat; background-position: right top} \r'
 'a:link {color:#000080text-decoration: underline cursor: hand}\r'
 'a:visited,a:active {color:#0080C0text-decoration: underline cursor: hand}\r'
 'a:hover{color:rgb(255,93,0);text-decoration:none}\r'
 'td {font-size:12pt}\r'
 'select {font-size:9pt;font-color:#0033CC;line-height:140%}\r'
 '.head {font-size:9pt}\r'
 '.foot-chn {font-size:9pt;line-height:120%}\r'
 ".foot-eng {font-size:9px;font-family:'verdana'; letter-spacing:2px}\r"
 '.adduser  {font-size:9pt;line-height:120%}\r'
 '-->\r'
 '</style>\r'
 '<table width="100%"

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,114,1,財管碩,FM611E,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,封之遠,"四2,3,4(管CM 2026)",,,,234,,,,《獨立研究》
1,114,1,財管碩,FM611F,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,吳藺芝,"四A,1,B(管CM 1025)",,,,A1B,,,,《獨立研究》
2,114,1,財管碩,FM611G,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,蔡秉真,"四2,3,4(管CM 1038)",,,,234,,,,《獨立研究》
3,114,1,財管碩,FM611H,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,鄭義,"一2,3,4(管CM 2020)",234,,,,,,,《獨立研究》
4,114,1,財管碩,FM611I,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,蔡維哲,"五A,1,2(管CM 1038)",,,,,A12,,,《獨立研究》
5,114,1,財管碩,FM611J,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,蔡佳芬,"二A,1,B(管CM 2020)",,A1B,,,,,,《獨立研究》
6,114,1,財管碩,FM611K,2,不分班,專題研究（一）INDEPENDENT STUDIES(I),3,學期,選修,...,羅勝議,"二A,1,B(管CM 2026)",,A1B,,,,,,《獨立研究》
7,114,1,財管碩,FM615,1,不分班,巨量資料分析導論INTRODUCTION TO BIG DATA ANALYTICS,3,學期,選修,...,蕭珮婕,"一2,3,4(管CM 1025)",234,,,,,,,《講授類》 金融科技學程人工智能投資微學程金融科技與創新微學程
8,114,1,財管碩,FM617,1,不分班,金融投資與程式交易FINANCIAL INVESTING AND PROGRAM TRADING,3,學期,選修,...,王昭文,"四5,6,7(管CM 3051)",,,,567,,,,《講授類》『本課程之限修，於加退選2階段開放選修』。限財管碩、金融創新產業碩士專班及財管碩士...
9,114,1,財管碩,FM618,1,不分班,營運資金管理決策WORKING CAPITAL MANAGEMENT DECISION,3,學期,選修,...,張世賢,"二2,3,4(管CM 2015)",,234,,,,,,《講授類》 財務管理專業學程


In [16]:
# 彙整結果
output = pd.concat([output, iOutput], ignore_index=True)
output

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,114,1,財管碩,FM506,1,不分班,金融市場理論與實務FINANCIAL MARKETS THEORY AND PRACTICE,3,學期,選修,...,邱敬貿,"一8,6,7(管CM 1023)",678,,,,,,,《講授類》 金融工程學程金融科技學程財務管理專業學程
1,114,1,財管碩,FM510,1,不分班,財務管理理論FINANCIAL MANAGEMENT THEORIES,3,學期,選修,...,廖源星,"一2,3,4(管CM 3019)",234,,,,,,,《講授類》 金融科技學程財務管理專業學程
2,114,1,財管碩,FM517,1,不分班,現代投資組合理論與策略MODERN PORTFOLIO THEORY AND INVESTM...,3,學期,選修,...,"蕭珮婕,鄭義","四2,3,4(管CM 1025)",,,,234,,,,《講授類》 金融工程學程金融科技學程人工智能投資微學程財務管理專業學程
3,114,1,財管碩,FM521,1,不分班,財務研究方法（一）METHODOLOGY FOR FINANCE (I),3,學期,選修,...,佘博文,"五2,3,4(管CM 2037)",,,,,234,,,《講授類》 金融科技學程
4,114,1,財管碩,FM527,1,不分班,證券市場微結構實務SECURITIES MARKETS MICROSTRUCTURE PRA...,3,學期,選修,...,"邱敬貿,馬黛","三5,6,7(管CM 2020)",,,567,,,,,《講授類》 金融工程學程金融科技學程金融科技與創新微學程
5,114,1,財管碩,FM532,1,不分班,財務報表分析與權益評價FINANCIAL STATEMENT ANALYSIS AND EQ...,3,學期,選修,...,吳藺芝,"二8,6,7(管CM 3017)",,678,,,,,,《講授類》 ※英語授課
6,114,1,財管碩,FM537,1,不分班,投資銀行與法金實務INVESTMENT BANKING AND CORPORATE FINA...,3,學期,選修,...,"翁培師,蔡秉真","二5,6,7(管CM 3019)",,567,,,,,,《講授類》
7,114,1,財管碩,FM541,1,不分班,行為財務學的應用APPLICATIONS OF BEHAVIORAL FINANCE,3,學期,選修,...,蔡佳芬,"三2,3,4(管CM 3022)",,,234,,,,,《講授類》 ※英語授課
8,114,1,財管碩,FM548,1,不分班,企業社會責任CORPORATE SOCIAL RESPONSIBILITY,3,學期,選修,...,封之遠,"四5,6,7(管CM 2026)",,,,567,,,,《講授類》
9,114,1,財管碩,FM551,1,不分班,永續投資與綠色金融創新SUSTAINABLE INVESTMENT AND GREEN FI...,3,學期,選修,...,封之遠,"五8,6,7(管CM 2015)",,,,,678,,,《講授類》
